In [ ]:
# import libraries
import netCDF4 as nc
import numpy as np
import glob as glob
import xarray as xr
import numpy.ma as ma
import matplotlib.pyplot as plt
from scipy.stats import percentileofscore
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta
import math
# read the ncfile
f1obs = sorted(glob.glob(r'era5regrid*monthly.nc'))
print(f1obs)
f1obsc = nc.Dataset(f1obs[0])
f2=nc.Dataset(f1obs[0])
tmaxd1obs=f1obsc.variables['d2m']
print(tmaxd1obs)
tmaxt1obs=f1obsc.variables['t2m']
print(tmaxt1obs)
tmaxd2obs=tmaxd1obs[:,:,:]-273.15
tmaxt2obs=tmaxt1obs[:,:,:]-273.15
dtwom1=tmaxd2obs
ttwom1=tmaxt2obs
expcomp1=((17.625*dtwom1)/(243.04+dtwom1))
expcomp2=((17.625*ttwom1)/(243.04+ttwom1))
ed=np.exp(expcomp1)
ea=np.exp(expcomp2)
relhum=100*(ed/ea)
expcomp3=((17.27*ttwom1)/(237.7+ttwom1))
vaprcmp1=np.exp(expcomp3)
vapr=(relhum/100)*6.105*vaprcmp1
wbgt1=0.567*ttwom1+0.393*vapr+3.94
wbgt1.dump('array2')
f1hssm = sorted(glob.glob(r'*Earth3regrid*historical*.nc'))
f1ftsm = sorted(glob.glob(r'*Earth3regrid*ssp585*.nc'))
f1hssmsc1 = nc.Dataset(f1hssm[0])
f1hssmsc2 = nc.Dataset(f1hssm[1])
f1hssmsc3 = nc.Dataset(f1hssm[2])
f1hssmsc4 = nc.Dataset(f1hssm[3])

f1ftsmsc1 = nc.Dataset(f1ftsm[0])
f1ftsmsc2 = nc.Dataset(f1ftsm[1])
f1ftsmsc3 = nc.Dataset(f1ftsm[2])
f1ftsmsc4 = nc.Dataset(f1ftsm[3])

hur1h=f1hssmsc1.variables['hur']
huss1h=f1hssmsc2.variables['huss']
ps1h=f1hssmsc3.variables['ps']
tas1h=f1hssmsc4.variables['tas']

hur1f=f1ftsmsc1.variables['hur']
huss1f=f1ftsmsc2.variables['huss']
ps1f=f1ftsmsc3.variables['ps']
tas1f=f1ftsmsc4.variables['tas']
huss2f=huss1f[:]
ps2f=ps1f[:]
tas2f=tas1f[:]
tas2fcel=tas2f-273.15

huss2f.shape, ps2f.shape, tas2f.shape, tas2fcel.shape

huss2f=huss1h[:]
ps2h=ps1h[:]
tas2h=tas1h[:]
tas2hcel=tas2h-273.15

huss2h.shape, ps2h.shape, tas2h.shape, tas2hcel.shape

vp1h=(huss2h*ps2h)/(0.622+huss2h)
vp1f=(huss2f*ps2f)/(0.622+huss2f)

wbgt2h=(0.567*tas2hcel)+(0.393*vp1h*0.01)+3.94
wbgt2f=(0.567*tas2fcel)+(0.393*vp1f*0.01)+3.94

wbgt1.shape, wbgt2h.shape, wbgt2f.shape

lon=f1obsc.variables['longitude']
print(lon)
lat=f1obsc.variables['latitude']
print(lat)

def getclosest_ij(lats,lons,latpt,lonpt):
    # find squared distance of every point on grid
    lat_diff = np.abs(lats - latpt)
    lon_diff = np.abs(lons - lonpt)
    lat_idx = np.argmin(lat_diff)
    lon_idx = np.argmin(lon_diff)
    nearest_lat = latvals[lat_idx]
    nearest_lon = lonvals[lon_idx]
    # Get 2D index for latvals and lonvals arrays from 1D index
    return nearest_lat,nearest_lon, lat_idx, lon_idx

latvals = lat[:]; lonvals = lon[:]
latuser=43.28
lonuseradjust=-105.82
if lonuseradjust<0:
    lonuser=360+lonuseradjust
else:
    lonuser=lonuseradjust
print(latuser,lonuser)

x1, y1, x2, y2 = getclosest_ij(latvals, lonvals, latuser, lonuser)
# print(x1)
# print(y1)
print("latidx",x2)
print("lonidx",y2)

nearest_lat = latvals[x2]
nearest_lon = lonvals[y2]
nearest_lat, nearest_lon

tmax2obs1=wbgt1[:,x2,y2]
tmax3hismodc1=wbgt2h[:,x2,y2]
tmax2fut1=wbgt2f[:,x2,y2]

tmax3obs = ma.filled(tmax2obs1, np.nan)
tmax4hismodc = ma.filled(tmax3hismodc1, np.nan)
tmax3fut = ma.filled(tmax2fut1, np.nan)

observed_flat = tmax3obs
his_model_flat = tmax4hismodc
fut_model_flat = tmax3fut

observed_flatwonan=observed_flat[~np.isnan(observed_flat)]
his_model_flatwonan=his_model_flat[~np.isnan(his_model_flat)]
fut_model_flatwonan=fut_model_flat[~np.isnan(fut_model_flat)]

model_present_corrected = np.zeros(his_model_flatwonan.size)  
model_future_corrected = np.zeros(fut_model_flatwonan.size)

for ival, model_value in enumerate(his_model_flatwonan):
    percentile = percentileofscore(his_model_flatwonan, model_value)
    model_present_corrected[ival] = np.percentile(observed_flatwonan, percentile)
print("Done1")

for ival, model_value in enumerate(fut_model_flatwonan):
    percentile = percentileofscore(fut_model_flatwonan, model_value)
    model_future_corrected[ival] = model_value + np.percentile(observed_flatwonan, percentile) - np.percentile(his_model_flatwonan, percentile)
print("Done2")

# Sort the data
sorted_data1 = np.sort(observed_flat)
sorted_data2 = np.sort(his_model_flatwonan)
sorted_data3 = np.sort(fut_model_flatwonan)
# Compute the CDF
cdf1 = np.arange(1, len(sorted_data1) + 1) / len(sorted_data1)
cdf2 = np.arange(1, len(sorted_data2) + 1) / len(sorted_data2)
cdf3 = np.arange(1, len(sorted_data3) + 1) / len(sorted_data3)

# Plot the CDFs
plt.figure(figsize=(8, 5))
plt.plot(sorted_data1, cdf1, label='Dataset 1')
plt.plot(sorted_data2, cdf2, label='Dataset 2')
plt.plot(sorted_data3, cdf3, label='Dataset 3')


plt.title('Cumulative Distribution Function (CDF) of Multiple Datasets')
plt.xlabel('Data Values')
plt.ylabel('Cumulative Probability')
plt.legend()
plt.grid(True)
plt.show()

# Sort the data
sorted_data1 = np.sort(observed_flat)
sorted_data2 = np.sort(model_present_corrected)
sorted_data3 = np.sort(model_future_corrected)
# Compute the CDF
cdf1 = np.arange(1, len(sorted_data1) + 1) / len(sorted_data1)
cdf2 = np.arange(1, len(sorted_data2) + 1) / len(sorted_data2)
cdf3 = np.arange(1, len(sorted_data3) + 1) / len(sorted_data3)

# Plot the CDFs
plt.figure(figsize=(8, 5))
plt.plot(sorted_data1, cdf1, label='Dataset 1')
plt.plot(sorted_data2, cdf2, label='Dataset 2')
plt.plot(sorted_data3, cdf3, label='Dataset 3')

plt.title('Cumulative Distribution Function (CDF) of Multiple Datasets')
plt.xlabel('Data Values')
plt.ylabel('Cumulative Probability')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
days=np.arange(1, 13, 1)
threshold_temperature=26
model_future_correctedplt=model_future_corrected[1:13]
plt.plot(days, model_future_correctedplt, label='Daily Temperature (°C)')
# Add a horizontal line for the threshold temperature
plt.axhline(y=threshold_temperature, color='r', linestyle='--', label=f'Threshold: {threshold_temperature}°C')

plt.title('Daily maximum temperature for year 2050')
plt.xlabel('Days')
plt.ylabel('Temperature')
plt.legend()
plt.grid(True)
plt.show()

# Count the number of days above the threshold
daily_temperatures=model_future_corrected
days_above_threshold = np.sum(daily_temperatures > threshold_temperature)

# Display the result
print(f"Number of days above {threshold_temperature}°C: {days_above_threshold} days")
